In [1]:
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [2]:
##extracting anticoag info as requested
sql = '''
SELECT
  extract(year from month) AS year,
  month,
  bnf_code AS BNFCode,
  bnf_name AS BNFName,
  SUM(items) AS TotalItems,
  SUM(actual_cost) AS TotalCost  
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
WHERE
  bnf_code LIKE "0208020Y0%" ##rivaroxaban
  OR bnf_code LIKE "0208020X0%" ##dabigatran
  OR bnf_code LIKE "0208020Z0%" ##apixaban
  OR bnf_code LIKE "0208020AA%" ##edoxaban
  OR bnf_code LIKE "0208020V0%" ##warfarin
  OR bnf_code LIKE "0208020H0%" ##acenocoumarol
  OR bnf_code LIKE "0208020N0%" ##phenidione
GROUP BY
  year,
  month,
  bnf_code,
  bnf_name
  '''

df_anticoag = bq.cached_read(sql, csv_path='anticoag.zip')

In [3]:
df_anticoag.head()

,year,month,BNFCode,BNFName,TotalItems,TotalCost
0,2012,2012-09-01,0208020Z0AAAAAA,Apixaban_Tab 2.5mg,2,88.74
1,2011,2011-01-01,0208020N0AAAAAA,Phenindione_Tab 10mg,885,74933.16
2,2014,2014-11-01,0208020Z0AAAAAA,Apixaban_Tab 2.5mg,8938,403476.98
3,2013,2013-11-01,0208020V0AAAMAM,Warfarin Sod_Liq Spec 3mg/5ml,2,310.52
4,2013,2013-01-01,0208020H0BBAAAA,Sinthrome_Tab 1mg,1278,5811.70


In [4]:
## teh requester wants a denominator of all "eligible population". 
##We don't know what this is but we can provide an age breakdown but perhaps they may wish to exclude children. 
##we can suggest looking at QoF etc as well
sql = ''' 
SELECT 
extract(year from month) AS year,
month,
sum(male_0_4) as male_0_4,
sum(male_5_14) as m_5_14,
sum(male_15_24) as m_15_24,
sum(male_25_34) as m_25_34,
sum(male_35_44) as m_35_44,
sum(male_45_54) as m_45_54,
sum(male_55_64) as m_55_64,
sum(male_65_74) as m_65_74,
sum(male_75_plus) as m_75_plus,
sum(female_0_4) as f_0_4,
sum(female_5_14) as f_5_14,
sum(female_15_24) as f_15_24,
sum(female_25_34) as f_25_34,
sum(female_35_44) as f_35_44,
sum(female_45_54) as f_45_54,
sum(female_55_64) as f_55_64,
sum(female_65_74) as f_65_74,
sum(female_75_plus) as f_75_plus,
sum(total_list_size) as total_list_size
FROM `ebmdatalab.hscic.practice_statistics` 
GROUP BY
year,
month
'''

df_praclist = bq.cached_read(sql, csv_path='praclist.zip')
df_praclist.head()

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,year,month,male_0_4,m_5_14,m_15_24,m_25_34,m_35_44,m_45_54,m_55_64,m_65_74,...,f_0_4,f_5_14,f_15_24,f_25_34,f_35_44,f_45_54,f_55_64,f_65_74,f_75_plus,total_list_size
0,2015,2015-06-01,1735883,3312576,3522304,4070736,3961024,4146074,3197793,2579726,...,1651229,3159398,3521580,4104607,3745266,3967579,3168360,2742252,2575527,57024156
1,2017,2017-08-01,1699943,3498031,3463865,4223416,4033147,4225103,3391995,2705978,...,1615555,3333009,3462354,4257073,3810526,4031307,3339214,2871948,2628121,58538696
2,2014,2014-05-01,1741315,3251083,3536619,4030399,3946135,4103343,3153685,2521103,...,1656739,3100036,3528684,4044709,3726751,3922923,3131373,2679809,2556274,56455131
3,2018,2018-09-01,1679056,3566541,3474229,4301494,4108993,4225284,3497355,2737953,...,1595641,3398174,3473303,4318124,3874266,4019389,3434376,2904127,2677767,59297331
4,2017,2017-10-01,1696284,3514486,3475969,4239342,4047663,4228673,3408025,2711504,...,1612802,3348689,3480768,4270809,3822378,4032640,3353985,2876845,2639118,58719921
